# Summary

This example demonstrates how to write simple Python code that calls an LLM API (OpenAI in this case) and processes the model’s response. It covers two approaches:
1. Using the OpenAI Python SDK
2. Using LangChain’s ChatOpenAI class

## Prerequisites

You can either use the paid OpenAI API platform, or run an open-source OpenAI model such as `gpt-oss-20b` locally through LM Studio (the method used in this example).

In [2]:
%pip install langchain langchain-core langchain-community

  Using cached langchain_community-0.4.1-py3-none-any.whl.metadata (3.0 kB)
  Using cached langchain_classic-1.0.0-py3-none-any.whl.metadata (3.9 kB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached pydantic_settings-2.12.0-py3-none-any.whl.metadata (3.4 kB)
  Using cached httpx_sse-0.4.3-py3-none-any.whl.metadata (9.7 kB)
  Using cached marshmallow-3.26.1-py3-none-any.whl.metadata (7.3 kB)
  Using cached typing_inspect-0.9.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached langchain_text_splitters-1.0.0-py3-none-any.whl.metadata (2.6 kB)
Using cached langchain_community-0.4.1-py3-none-any.whl (2.5 MB)
Using cached dataclasses_json-0.6.7-py3-none-any.whl (28 kB)
Using cached httpx_sse-0.4.3-py3-none-any.whl (9.0 kB)
Using cached langchain_classic-1.0.0-py3-none-any.whl (1.0 MB)
Using cached langchain_text_splitters-1.0.0-py3-none-any.whl (33 kB)
Using cached marshmallow-3.26.1-py3-none-any.whl (50 kB)
Using cached pydantic_settings-2.12.0-py3-none

In [3]:
%pip install -U langchain-openai

  Using cached langchain_openai-1.1.0-py3-none-any.whl.metadata (2.6 kB)
Using cached langchain_openai-1.1.0-py3-none-any.whl (84 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 995.8/995.8 kB 16.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [langchain-openai]
Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install openai

Note: you may need to restart the kernel to use updated packages.


In [5]:
!python --version

Python 3.10.19


In [8]:
%pip show openai

Name: openai
Version: 2.8.1
Summary: The official Python library for the openai API
Home-page: https://github.com/openai/openai-python
Author: 
Author-email: OpenAI <support@openai.com>
License: Apache-2.0
Location: /opt/homebrew/anaconda3/lib/python3.11/site-packages
Requires: anyio, distro, httpx, jiter, pydantic, sniffio, tqdm, typing-extensions
Required-by: 
Note: you may need to restart the kernel to use updated packages.


## Examples

### Use the OpenAI library directly
The example below uses the OpenAI Python SDK to interact with a locally hosted open-weight reasoning model.

In [14]:
# Method 1 - Use OpenAI Library

from openai import OpenAI
import os

messages = [
    {"role": "user", "content": "Introduce yourself"}
]

client = OpenAI(
    api_key = "Local Server, no key",
    base_url = "http://127.0.0.1:1234/v1",
)

completion = client.chat.completions.create(
    model = "gpt-oss-20b",
    temperature = 0,
    messages = messages,
    stream = False
)

print(completion.choices[0].message.content)

Hello! I’m ChatGPT, a large language model created by OpenAI. I can help answer questions, brainstorm ideas, explain concepts, write stories or code, and chat about almost any topic you’re interested in. I’m here to assist, learn from our conversation, and make your experience as helpful and engaging as possible. How can I help you today?


### Use LangChain
The example below uses LangChain to build a simple chain that chats with a locally hosted model, instructs it to return comma-separated values, and then parses the output into a list.

In [17]:
# Method 2 - Use LangChain

import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import CommaSeparatedListOutputParser

llm = ChatOpenAI(
    model = "gpt-oss-20b",
    api_key = "Local Server, no key",
    base_url = "http://127.0.0.1:1234/v1",
    verbose = False
)

prompt = PromptTemplate.from_template(
    "Give me 5 popular names in {country} for {gender}. Return the result in comma separated, no other text."
)

chain = prompt | llm | CommaSeparatedListOutputParser()

response = chain.invoke({"country": "China", "gender": "male" })

print(f"Proposed names: {response}")

Proposed names: ['Wei', 'Jun', 'Ming', 'Hao', 'Jie']


### Use LangChain to parse the response into a Pydantic object
Use `PydanticOutputParser` to validate the model’s output and convert it into a structured Pydantic object.

In [ ]:
from pydantic import BaseModel
from langchain_core.prompts import load_prompt
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI # Or your preferred LLM

# Define your Pydantic model (as shown above)
class Person(BaseModel):
    name: str
    age: int

# Initialize the output parser
parser = PydanticOutputParser(pydantic_object=Person)

print(parser.get_format_instructions())

# Load the prompt
# If the output_parser is defined in the prompt file:
# prompt = load_prompt("prompt.yaml")
# If not, you need to add format instructions to the prompt:
prompt = load_prompt("prompt_without_parser_in_file.yaml") # Using the second example prompt above

# Create an LLM instance
llm = ChatOpenAI(
    model = "gpt-oss-20b",
    api_key = "Local Server, no key",
    base_url = "http://127.0.0.1:1234/v1",
    temperature=0
)

# Create a chain
chain = (
    {"text": RunnablePassthrough(), "format_instructions": lambda x: parser.get_format_instructions()}
    | prompt
    | llm
    | parser
)

# Invoke the chain
result = chain.invoke("You can call me Alice, and I was born 30 years ago")
print(result)
# Expected output: Person(name='Alice', age=30)

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"name": {"title": "Name", "type": "string"}, "age": {"title": "Age", "type": "integer"}}, "required": ["name", "age"]}
```
name='Alice' age=30


In [ ]:
from langchain_core.prompts import FewShotPromptTemplate
from langchain_core.example_selectors.length_based import LengthBasedExampleSelector
from langchain_openai import ChatOpenAI # Or your preferred LLM

# Create an LLM instance
llm = ChatOpenAI(
    model = "gpt-oss-20b",
    api_key = "Local Server, no key",
    base_url = "http://127.0.0.1:1234/v1",
    temperature=0,
    verbose=False,
)

# 50 examples
examples = [
  {"input": "mm", "output": "1 mm = 0.1 cm"},
  {"input": "m", "output": "1 m = 100 cm"},
  {"input": "km", "output": "1 km = 1000 m"},
  {"input": "in", "output": "1 in = 2.54 cm"},
  {"input": "ft", "output": "1 ft = 12 in"},
  {"input": "yd", "output": "1 yd = 3 ft"},
  {"input": "mi", "output": "1 mi = 5280 ft"},
  {"input": "g", "output": "1 g = 1000 mg"},
  {"input": "kg", "output": "1 kg = 1000 g"},
  {"input": "mg", "output": "1 mg = 0.001 g"},
  {"input": "lb", "output": "1 lb = 16 oz"},
  {"input": "oz", "output": "1 oz = 28.35 g"},
  {"input": "l", "output": "1 L = 1000 mL"},
  {"input": "ml", "output": "1 mL = 0.001 L"},
  {"input": "gal", "output": "1 gal = 3.785 L"},
  {"input": "qt", "output": "1 qt = 2 pt"},
  {"input": "pt", "output": "1 pt = 2 cups"},
  {"input": "cup", "output": "1 cup = 16 tbsp"},
  {"input": "tbsp", "output": "1 tbsp = 3 tsp"},
  {"input": "tsp", "output": "1 tsp = 5 mL"},
  {"input": "s", "output": "1 s = 1000 ms"},
  {"input": "ms", "output": "1 ms = 0.001 s"},
  {"input": "min", "output": "1 min = 60 s"},
  {"input": "hr", "output": "1 hr = 60 min"},
  {"input": "day", "output": "1 day = 24 hr"},
  {"input": "n", "output": "1 N = 1 kg·m/s²"},
  {"input": "pa", "output": "1 Pa = 1 N/m²"},
  {"input": "kpa", "output": "1 kPa = 1000 Pa"},
  {"input": "mpa", "output": "1 MPa = 1000 kPa"},
  {"input": "j", "output": "1 J = 1 N·m"},
  {"input": "kj", "output": "1 kJ = 1000 J"},
  {"input": "w", "output": "1 W = 1 J/s"},
  {"input": "kw", "output": "1 kW = 1000 W"},
  {"input": "c", "output": "1°C = 33.8°F"},
  {"input": "f", "output": "1°F = -17.22°C"},
  {"input": "k", "output": "1 K = -272.15°C"},
  {"input": "bps", "output": "1 bps = 1 bit/s"},
  {"input": "kbps", "output": "1 kbps = 1000 bps"},
  {"input": "mbps", "output": "1 Mbps = 1000 kbps"},
  {"input": "gbps", "output": "1 Gbps = 1000 Mbps"},
  {"input": "byte", "output": "1 byte = 8 bits"},
  {"input": "kb", "output": "1 KB = 1024 bytes"},
  {"input": "mb", "output": "1 MB = 1024 KB"},
  {"input": "gb", "output": "1 GB = 1024 MB"},
  {"input": "tb", "output": "1 TB = 1024 GB"},
  {"input": "rad", "output": "1 rad = 57.2958°"},
  {"input": "deg", "output": "1° = 0.01745 rad"},
  {"input": "atm", "output": "1 atm = 101.325 kPa"},
  {"input": "bar", "output": "1 bar = 100 kPa"}
]

example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Word: {input}\nExplanation:{output}",
)

example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    max_length=100,         # the length here is number of bytes
)

prompt = FewShotPromptTemplate(
    example_selector = example_selector,
    example_prompt = example_prompt,
    suffix="Word:{input}\nReturn the explanation for this word and show nothing else",
    input_variables=["input"],
)
print(prompt.format(input="gigawatt"))

result = llm.invoke(prompt.format(input="gigawatt"))
print(result.content)

Word: mm
Explanation:1 mm = 0.1 cm

Word: m
Explanation:1 m = 100 cm

Word: km
Explanation:1 km = 1000 m

Word: in
Explanation:1 in = 2.54 cm

Word: ft
Explanation:1 ft = 12 in

Word: yd
Explanation:1 yd = 3 ft

Word: mi
Explanation:1 mi = 5280 ft

Word: g
Explanation:1 g = 1000 mg

Word: kg
Explanation:1 kg = 1000 g

Word: mg
Explanation:1 mg = 0.001 g

Word: lb
Explanation:1 lb = 16 oz

Word: oz
Explanation:1 oz = 28.35 g

Word: l
Explanation:1 L = 1000 mL

Word: ml
Explanation:1 mL = 0.001 L

Word:gigawatt
Return the explanation for this word and show nothing else
content='1 gigawatt = 10⁹ watts' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 255, 'prompt_tokens': 282, 'total_tokens': 537, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_provider': 'openai', 'model_name': 'gpt-oss-20b', 'system_fingerprint': 'gpt-oss-20b', 'id': 'chatcmpl-enhjf232s36eimksvg7yb', 'finish_reason': 'stop', 'logprobs': None} id='lc_ru